## Unsupervised Ensemble Learning

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D, get_test_data
from matplotlib import cm
from time import time
from itertools import permutations, product

### Auxiliary Functions

In [2]:
normP = lambda x: np.maximum(x, 1e-9)/np.sum(np.maximum(x, 1e-9))
normc = lambda x: x/np.linalg.norm(x)
str_arr = lambda arr: " ".join("%.2f"%x for x in arr)
str_mat = lambda mat: "\n".join(str_arr(arr) for arr in mat) 
def normRowP(A, dim = 0):
    #norm P across dim: 0: cols, 1: rows
    r,c = A.shape
    if dim == 0: 
        rep = (r,1)
    elif dim == 1:
        rep = (1,c)
    else:
        return None
        
    arr = np.sum(A, axis = dim)
    A_new = np.divide(A, np.tile(arr, (2,1)))
    
def genConfMat(k, alpha):
    # k is the size of the confusion matrix
    # alpha is the lower limit for the diagonal entry
    C = alpha*np.eye(k) + (1-alpha)*np.array([normP(1 + np.random.rand(k)) for i in range(k)])
    return C
def tensorProdk(a,k):
    res = a
    for _ in range(k-1):
        res = np.tensordot(res, a, axes = 0)
    return res

def tensorwhiten(T,W):
    # T(W,W,W)
    I,J,K = T.shape
    T_new = np.zeros((I,J,K))
    for i,j,k in product(range(I), range(J), range(K)):
        T_new[i,j,k] = np.sum([ T[i_p,j_p,k_p]*W[i_p,i]*W[j_p,j]*W[k_p,k] for i_p,j_p,k_p in product(range(I), range(J), range(K))])
    return T_new

def tensormult2(T, v):
    # T(I,v,v)
    I,J,K = T.shape
    v_new = np.zeros(I)
    for i in range(I):
        v_new[i] = np.sum([T[i,j_p,k_p]*v[j_p]*v[k_p] for j_p,k_p in product(range(J), range(K))])
    return v_new

def tensormult3(T, v):
    # T(v,v,v)
    I,J,K = T.shape
    val = np.sum([T[i_p,j_p,k_p]*v[i_p]*v[j_p]*v[k_p] for i_p,j_p,k_p in product(range(I), range(J), range(K))])
    return val

### Classifier Ensemble Class

In [212]:
class classifierEnsemble:   
    def __init__(self, params):
        self.ph_true = params['ph']
        self.num_classifier = params['num_classifier']
        self.conf_true = params['conf'] # dictionary of confusion matrices
        # each row: one true class 
        self.num_class = len(self.ph_true) #number of classes
        self.ph_est = np.zeros(self.num_class) 
        self.conf_est = {i: np.zeros((self.num_class,self.num_class)) for i in range(self.num_classifier)}
    
    def genData(self, n, adv_map, adv_p, seed = None):
        if seed is not None:
            np.random.seed(seed)
        # fraction of adversarial examples
        adv_p = min(1, max(0, adv_p))
        # adversarial examples
        adv_arr = np.random.choice([False, True], p = [1-adv_p, adv_p], size = n)
        # true labels
        h_arr = np.random.choice(range(self.num_class), p = self.ph_true, size = n)
        # labels from the classifiers
        labels_arr = np.zeros((n,self.num_classifier, self.num_class))
        for i in range(n):
            h = h_arr[i]
            if adv_arr[i]: #adversarial Example
                labels_arr[i,:,adv_map[h]] = 1
            else:
                for j in range(self.num_classifier):
                    l = np.random.choice(range(self.num_class), p = self.conf_true[j][h,:])
                    labels_arr[i,j,l] = 1
        return adv_arr, h_arr, labels_arr

### Instantiate ensemble of classifiers (Start  of the Experiment)

In [413]:
num_classifier = 20
num_class = 10
ph_true = np.random.rand(num_class); ph_true = ph_true/np.sum(ph_true);
alpha_arr = np.minimum(1, np.array([0.2, 0.05, 0.0])+1.0/num_class) #minimum accuracy per worker
alpha_p = [0.5, 0.3, 0.2] # probability of choosing alpha from alpha_arr
alpha = np.random.choice(alpha_arr, p = alpha_p, size = num_classifier)
print "Alpha:%s"%str_arr(alpha)
conf_dict = {i: genConfMat(num_class, alpha[i]) for i in range(num_classifier)}
E = classifierEnsemble({'ph': ph_true, 'num_classifier': num_classifier, 'conf': conf_dict})

Alpha:0.30 0.15 0.15 0.15 0.15 0.15 0.10 0.10 0.30 0.15 0.30 0.30 0.15 0.15 0.10 0.30 0.30 0.10 0.10 0.30


### Partition classifiers in three groups

In [389]:
g = {}
while len(g.keys()) < 3:
    groups = np.random.choice([0,1,2], p = [1/3.0, 1/3.0, 1/3.0], size = num_classifier)
    g ={i: list(groups ==i) for i in range(3) if np.sum(groups ==i) > 0}

### Computing true parameters

In [390]:
mu_true = {j: np.mean([conf_dict[i] for i,f in enumerate(g[j]) if f],axis = 0) for j in range(3)}
M2_true = {}; M3_true = {}
for j in range(3):
    M = np.zeros((num_class, num_class))
    T = np.zeros((num_class, num_class, num_class))
    for i in range(num_class):
        M += ph_true[i]*tensorProdk(mu_true[j][i],2)
        T += ph_true[i]*tensorProdk(mu_true[j][i],3)
    M2_true[j] = M
    M3_true[j] = T

### Data generation and aggregation

In [391]:
'''
Adversarial inputs 
--------------------
- This skews the estimation of confusion matrix.
- The classifiers are conditionally independent on real inputs.
'''
# Adversarial input parameters
#adv_map = np.random.permutation(range(num_class)) # map of adversarial labels
adv_map = np.mod(1+np.arange(num_class), num_class) # map of adversarial labels
adv_p = 0.1 # probability that an input is adversarial

In [392]:
num_data = 20000
## Data Format: 
# - labels[i,j,:] = one hot encoding of label for time i and classifier j 
# - true_label[i] = true label (categorical) for time i
# - adv[i] = 'True' if adversarial ip o/w 'False'
adv, true_label, labels = E.genData(num_data, adv_map, adv_p)
# test set
adv_test, true_label_test, labels_test = E.genData(num_data, adv_map, adv_p)
# Data aggregation
Z ={i: np.mean(labels[:,g[i],:], 1) for i in range(3)}

### Convert to symmetric moments

In [393]:
perms = [(1,2,0), (0,1,2), (2,0,1)]
M2 = {}; M3 = {}; 
for p in perms:
    a, b, c = p
    Mcb = np.mean([np.tensordot(Z[c][j,:], Z[b][j,:], axes =0) for j in range(num_data)], axis = 0)
    Mab = np.mean([np.tensordot(Z[a][j,:], Z[b][j,:], axes =0) for j in range(num_data)], axis = 0)
    Za_prime = [np.dot(np.dot(Mcb, np.linalg.inv(Mab)), Z[a][j]) for j in range(num_data)]
    
    Mca = np.mean([np.tensordot(Z[c][j,:], Z[a][j,:], axes =0) for j in range(num_data)], axis = 0)
    Mba = np.mean([np.tensordot(Z[b][j,:], Z[a][j,:], axes =0) for j in range(num_data)], axis = 0)
    Zb_prime = [np.dot(np.dot(Mca, np.linalg.inv(Mba)), Z[b][j,:]) for j in range(num_data)]
    
    M2[c] = np.mean([np.tensordot(Za_prime[j], Zb_prime[j], axes =0) for j in range(num_data)], axis = 0)
    M3[c] = np.mean([np.tensordot(np.tensordot(Za_prime[j], Zb_prime[j], axes = 0), Z[c][j,:], axes = 0) for j in range(num_data)], axis = 0)

### Empirical moments vs True moments

In [394]:
for j in range(3):
    print 'Error M2_true[%d] vs M2_est[%d]:%.5f'%(j,j, np.linalg.norm(M2_true[j]-M2[j]))
    print 'Error M3_true[%d] vs M3_est[%d]:%.5f'%(j,j, np.linalg.norm(M3_true[j]-M3[j]))

Error M2_true[0] vs M2_est[0]:0.03036
Error M3_true[0] vs M3_est[0]:0.03113
Error M2_true[1] vs M2_est[1]:0.03048
Error M3_true[1] vs M3_est[1]:0.03286
Error M2_true[2] vs M2_est[2]:0.03076
Error M3_true[2] vs M3_est[2]:0.03314


### Recovering Group Confusion Matrices using Tensor Decomposition

In [395]:
# parameters
rank = num_class
Maxiter = 50; num_init = 5;
# initialiaztion
W ={}; mu_est = {}; ph_est = {}
disp = False
for j in range(3):
    c = perms[j][2] # group c parameters
    print '**Group:(%d)**'%c
    ## whitenning M2
    [U, l, V] = np.linalg.svd(M2[j]); 
    L = np.diag(l)
    W = np.dot( U[:, :rank], np.sqrt(np.linalg.inv(L[:rank, :rank])) );
    M3W = tensorwhiten(M3[j], W)
    ## tensor decomposition
    alpha_arr = np.zeros(rank)
    ph_est_int = np.zeros(rank)
    v_arr = np.zeros((rank, rank))
    mu_est_int = np.zeros((rank, rank))
    #------
    for i in range(rank): # loop over eigenvalues
        for t in range(num_init): # loop over initialization
            max_val = -1000;
            v_old = np.random.rand(rank); v_old = v_old/np.linalg.norm(v_old);
            for j in range(Maxiter): # power method iteration
                v_new = tensormult2(M3W, v_old); v_new = v_new/np.linalg.norm(v_new);
                v_old = v_new
            new_val = tensormult3(M3W, v_new)
            if new_val > max_val:
                alpha_arr[i] = new_val
                v_arr[:,i] = v_new
                max_val = new_val
        # Unwhittening the vectors are stacked into columns for mu_est_int
        ph_est_int[i] = alpha_arr[i]**(-2)
        mu_est_int[:,i] = normP(np.dot(np.linalg.inv(W.T), alpha_arr[i]*v_arr[:,i]))
        # deflate tensor
        M3W -= alpha_arr[i]*tensorProdk(v_arr[:,i],3)
        if disp: print 'Tensor norm after %d-th eigenpair:%.3f'%(i, np.linalg.norm(M3W))
    #--------
    print 'Recovered eigenpairs:\n%s'%"\n".join('%.2f:[%s]'%(alpha_arr[i], str_arr(v_arr[:,i])) for i in range(num_class))       
    if disp: print 'mu_est_int[%d]:\n%s'%(c,"\n".join("%s"%str_arr(mu_est_int[i,:]) for i in range(rank)))
    # finding the correct permutation of eigenpairs
    # - the l-th col with the highest l-th row entry becomes the new l-th the col 
    ind = [np.argmax(mu_est_int[i,:]) for i in range(num_class)]
    if disp: print 'Correct order:%s'%str_arr(ind)
    mu_est[c] = mu_est_int[:,ind] # rearranging the rows(?)
    ph_est[c] = ph_est_int[ind]
    #print 'ph_est:%s'%str_arr(ph_est[c])
    #print 'mu_est:\n%s'%"\n".join("%s"%str_arr(mu_est[c][:,i]) for i in range(rank))

**Group:(0)**
Recovered eigenpairs:
6.03:[-0.17 0.05 0.08 -0.10 0.02 0.09 -0.34 -0.75 -0.47 -0.19]
6.73:[-0.15 0.09 0.29 0.89 -0.19 -0.18 0.13 0.07 -0.07 0.03]
8.78:[-0.10 0.02 0.08 -0.02 0.05 0.05 -0.16 -0.30 0.65 0.66]
6.76:[-0.14 0.14 0.08 -0.07 0.22 0.76 0.56 0.05 -0.09 0.02]
6.32:[-0.15 0.18 0.20 -0.28 0.51 -0.65 0.34 0.12 -0.14 0.08]
5.65:[-0.17 0.03 -0.03 -0.05 0.11 -0.03 -0.21 0.19 0.57 -0.74]
6.26:[-0.16 0.21 -0.93 0.12 -0.07 -0.08 0.11 0.04 -0.12 0.08]
7.27:[-0.13 0.16 0.17 -0.41 -0.83 -0.11 0.20 0.10 -0.03 0.01]
5.89:[-0.18 -0.96 -0.10 0.01 -0.07 -0.05 0.14 0.03 -0.08 0.06]
7.41:[-0.12 0.05 0.10 -0.05 0.10 0.18 -0.59 0.64 -0.31 0.26]
**Group:(2)**
Recovered eigenpairs:
7.38:[-0.13 0.01 0.02 -0.11 -0.18 0.10 -0.71 -0.65 0.02 -0.07]
8.79:[-0.10 0.05 -0.01 -0.07 -0.03 -0.01 -0.13 0.36 -0.73 -0.55]
6.73:[-0.15 0.23 0.41 0.76 0.20 -0.25 0.14 -0.20 0.05 -0.11]
7.24:[-0.13 0.08 0.16 -0.24 -0.75 -0.31 0.45 -0.11 0.13 -0.07]
5.90:[-0.18 -0.92 -0.22 0.23 0.01 -0.04 0.10 -0.03 0.05 -0.

### Estimated vs True Group Confusion Matrices

In [396]:
print "Collective Confusion Matrix"
for j in range(3):
    print '**Group:(%d)**'%j
    print 'ph_est:%s'%str_arr(ph_est[j])
    print 'mu_est:\n%s'%str_mat(mu_est[j])
    print 'mu_true:\n%s'%str_mat(mu_true[j])
    print 'Error:%s'%np.linalg.norm(mu_true[j]- mu_est[j])
ph_est_avg = np.mean([ph_est[j] for j in range(3)], axis = 0)
print '**Avg ph_est:%s**'%str_arr(ph_est_avg)

Collective Confusion Matrix
**Group:(0)**
ph_est:0.03 0.02 0.01 0.03 0.03 0.02 0.02 0.03 0.03 0.02
mu_est:
0.78 0.02 0.02 0.07 0.00 0.03 0.02 0.03 0.01 0.01
0.03 0.89 0.01 0.03 0.02 0.01 0.03 0.05 0.03 0.02
0.02 0.01 0.84 0.04 0.02 0.02 0.00 0.04 0.00 0.02
0.05 0.01 0.02 0.61 0.00 0.02 0.02 0.03 0.02 0.02
0.00 0.00 0.02 0.06 0.85 0.04 0.03 0.04 0.02 0.01
0.02 0.02 0.03 0.03 0.03 0.83 0.03 0.03 0.02 0.02
0.03 0.00 0.01 0.07 0.01 0.03 0.82 0.04 0.01 0.03
0.02 0.03 0.02 0.00 0.02 0.00 0.00 0.64 0.02 0.04
0.02 0.01 0.00 0.07 0.03 0.01 0.01 0.03 0.85 0.05
0.03 0.02 0.03 0.02 0.01 0.01 0.05 0.05 0.02 0.79
mu_true:
0.24 0.09 0.08 0.09 0.09 0.08 0.08 0.08 0.09 0.09
0.09 0.23 0.08 0.09 0.08 0.09 0.08 0.09 0.09 0.09
0.09 0.09 0.25 0.07 0.09 0.09 0.08 0.09 0.08 0.09
0.09 0.07 0.08 0.24 0.09 0.09 0.09 0.08 0.09 0.08
0.08 0.09 0.08 0.08 0.23 0.09 0.09 0.09 0.09 0.08
0.09 0.08 0.08 0.09 0.09 0.24 0.08 0.08 0.08 0.09
0.08 0.08 0.08 0.09 0.09 0.08 0.23 0.08 0.09 0.09
0.09 0.08 0.09 0.09 0.08 0.09 0.09

### Estimating Individual Confusion Matrices

In [397]:
print "**Individual Confusion Matrix**"
C = {}
for j in range(3):
    c = perms[j][2]; a = perms[j][0];
    P1 = np.linalg.inv(np.dot(np.diag(ph_est_avg), mu_est[a].T))
    grp = [x for x in range(num_classifier) if g[c][x]]
    for i in grp: 
        P2_arr = [np.tensordot(labels[l,i,:], Z[a][l,:], axes = 0) for l in range(num_data)]
        P2 = np.mean(P2_arr, axis = 0)
        C[i] = np.dot(P2, P1)
        C[i] = np.array([normP(C[i][:,j]) for j in range(rank)])
err = {}
for i in range(num_classifier):
    print '---------------'
    print 'Classifier:',i
    print '---------------'
    print 'Estimate'
    print '%s'%"\n".join("%s"%str_arr(C[i][:,j]) for j in range(rank))
    print 'groundtruth'
    print '%s'%"\n".join("%s"%str_arr(conf_dict[i][:,j]) for j in range(rank))
    err[i] = np.linalg.norm(conf_dict[i]- C[i])
    print 'Error:%.2f'%err[i]

**Individual Confusion Matrix**
---------------
Classifier: 0
---------------
Estimate
0.24 0.08 0.08 0.08 0.08 0.07 0.09 0.08 0.08 0.08
0.09 0.27 0.08 0.09 0.07 0.08 0.08 0.08 0.07 0.08
0.07 0.08 0.21 0.09 0.08 0.08 0.08 0.08 0.07 0.09
0.08 0.08 0.08 0.19 0.08 0.09 0.09 0.09 0.08 0.08
0.08 0.08 0.09 0.10 0.31 0.08 0.09 0.09 0.07 0.09
0.08 0.07 0.08 0.09 0.07 0.26 0.08 0.08 0.08 0.08
0.08 0.08 0.10 0.09 0.08 0.08 0.24 0.10 0.08 0.09
0.09 0.09 0.10 0.10 0.08 0.09 0.10 0.22 0.09 0.09
0.09 0.08 0.09 0.09 0.07 0.08 0.08 0.08 0.29 0.08
0.08 0.08 0.09 0.08 0.08 0.08 0.09 0.09 0.08 0.25
groundtruth
0.27 0.10 0.09 0.09 0.06 0.06 0.10 0.06 0.10 0.07
0.09 0.28 0.09 0.08 0.10 0.07 0.10 0.08 0.06 0.07
0.06 0.08 0.27 0.10 0.11 0.09 0.07 0.07 0.07 0.09
0.07 0.07 0.06 0.27 0.08 0.08 0.11 0.06 0.10 0.06
0.06 0.05 0.09 0.10 0.27 0.10 0.07 0.08 0.07 0.10
0.10 0.06 0.06 0.06 0.08 0.28 0.06 0.09 0.11 0.06
0.09 0.07 0.11 0.07 0.09 0.07 0.26 0.10 0.08 0.11
0.09 0.10 0.07 0.09 0.08 0.06 0.08 0.30 0.08 0.10
0

### One-shot estimation of the labels using EM after spectral initialization

In [398]:
def compute_q_hat(labels, C):
    # for a particular time slot
    # labels[i,:]: the one hot encoding of label of classifier i
    # C: dictionary of confusion matrices
    arr = np.sum(np.dot(np.log(C[i]), labels[i,:]) for i in range(num_classifier))
    wghts = np.exp(arr-max(arr))
    q_hat = normP(wghts)
    return q_hat

In [406]:
# Parameters
EMiter = 20; batch_size = 2000; 
# Initialize
C_iter = {k:v for k,v in C.iteritems()}
# iteration
for ix in range(EMiter):
    samp_data = np.random.choice(num_data, min(batch_size, num_data))  # Full data
    true_label_iter = np.array([true_label[j] for j in samp_data])
    labels_iter = np.array([labels[j,:,:] for j in samp_data])
    # E-step
    q_hat_arr = np.array([compute_q_hat(labels[j,:,:], C_iter) for j in samp_data])
    # Estimate labels
    est_label = np.argmax(q_hat_arr, axis = 1)
    acc = np.mean(est_label== true_label_iter)
    print 'Accuracy after Spectral + %d EM steps:%.5f'%(ix,acc)
    # M-step 
    for i in range(num_classifier):
        mu_int = np.dot(q_hat_arr.T, labels_iter[:,i,:] )
        # normalize the vectors accross rows
        C_iter[i] = np.divide(mu_int.astype(float), mu_int.sum(axis = 1, keepdims = True)) 

Accuracy after Spectral + 0 EM steps:0.57850
Accuracy after Spectral + 1 EM steps:0.57600
Accuracy after Spectral + 2 EM steps:0.54850
Accuracy after Spectral + 3 EM steps:0.53350
Accuracy after Spectral + 4 EM steps:0.52150
Accuracy after Spectral + 5 EM steps:0.52750
Accuracy after Spectral + 6 EM steps:0.49600
Accuracy after Spectral + 7 EM steps:0.47650
Accuracy after Spectral + 8 EM steps:0.45550
Accuracy after Spectral + 9 EM steps:0.43800
Accuracy after Spectral + 10 EM steps:0.42200
Accuracy after Spectral + 11 EM steps:0.40350
Accuracy after Spectral + 12 EM steps:0.41400
Accuracy after Spectral + 13 EM steps:0.39850
Accuracy after Spectral + 14 EM steps:0.41400
Accuracy after Spectral + 15 EM steps:0.43300
Accuracy after Spectral + 16 EM steps:0.42000
Accuracy after Spectral + 17 EM steps:0.43000
Accuracy after Spectral + 18 EM steps:0.41850
Accuracy after Spectral + 19 EM steps:0.42100


In [407]:
# Parameters
EMiter = 20; batch_size = 2000; alpha_guess = 0.1;
# Initialize
C_iterEM = {k:genConfMat(rank, alpha_guess) for k in range(num_classifier)}
# iteration
for ix in range(EMiter):
    samp_data = np.random.choice(num_data, min(batch_size, num_data))  # Full data
    true_label_iter = np.array([true_label[j] for j in samp_data])
    labels_iter = np.array([labels[j,:,:] for j in samp_data])
    # E-step
    q_hat_arr = np.array([compute_q_hat(labels[j,:,:], C_iterEM) for j in samp_data])
    # Estimate labels
    est_label = np.argmax(q_hat_arr, axis = 1)
    acc = np.mean(est_label== true_label_iter)
    print 'Accuracy after %d-th EM steps:%.5f'%(ix,acc)
    # M-step 
    for i in range(num_classifier):
        mu_int = np.dot(q_hat_arr.T, labels_iter[:,i,:])
        # normalize the vectors accross rows
        C_iterEM[i] = np.divide(mu_int.astype(float), mu_int.sum(axis = 1, keepdims = True)) 

Accuracy after 0-th EM steps:0.53650
Accuracy after 1-th EM steps:0.60100
Accuracy after 2-th EM steps:0.56850
Accuracy after 3-th EM steps:0.58000
Accuracy after 4-th EM steps:0.56950
Accuracy after 5-th EM steps:0.54000
Accuracy after 6-th EM steps:0.54450
Accuracy after 7-th EM steps:0.51600
Accuracy after 8-th EM steps:0.51750
Accuracy after 9-th EM steps:0.49050
Accuracy after 10-th EM steps:0.44200
Accuracy after 11-th EM steps:0.45800
Accuracy after 12-th EM steps:0.43100
Accuracy after 13-th EM steps:0.41800
Accuracy after 14-th EM steps:0.40800
Accuracy after 15-th EM steps:0.41250
Accuracy after 16-th EM steps:0.38900
Accuracy after 17-th EM steps:0.39200
Accuracy after 18-th EM steps:0.41750
Accuracy after 19-th EM steps:0.40300


### Results: Final E-step and Estimation for the whole data 

In [408]:
real_test = [not x for x in adv_test]

#### 1) Spectral + EM

In [409]:
# E-step
q_hat_arr = np.array([compute_q_hat(labels_test[j,:,:], C_iter) for j in range(num_data)])
# Estimate labels
est_label = np.argmax(q_hat_arr, axis = 1)
acc_w_adv = np.mean(est_label == true_label_test)
acc_wo_adv = np.mean(est_label[real_test] == true_label_test[real_test])
print '**Final accuracy w/ adversarial i.p.(Spectral + %d EM steps):%.5f'%(i_best,acc_w_adv)
print '**Final accuracy w/o adversarial i.p.(Spectral + %d EM steps):%.5f'%(i_best,acc_wo_adv)

**Final accuracy w/ adversarial i.p.(Spectral + 12 EM steps):0.41865
**Final accuracy w/o adversarial i.p.(Spectral + 12 EM steps):0.46545


#### 2) Spectral

In [410]:
q_hat_arr = np.array([compute_q_hat(labels_test[j,:,:], C) for j in range(num_data)])
# Estimate labels
est_label = np.argmax(q_hat_arr, axis = 1)
acc_w_adv = np.mean(est_label == true_label_test)
acc_wo_adv = np.mean(est_label[real_test] == true_label_test[real_test])
print '**Final accuracy w/ adversarial i.p. (only Spectral):%.5f'%(acc_w_adv)
print '**Final accuracy w/o adversarial i.p. (only Spectral):%.5f'%(acc_wo_adv)

**Final accuracy w/ adversarial i.p. (only Spectral):0.59680
**Final accuracy w/o adversarial i.p. (only Spectral):0.66352


#### 3) EM

In [411]:
q_hat_arr = np.array([compute_q_hat(labels_test[j,:,:], C_iterEM) for j in range(num_data)])
# Estimate labels
est_label = np.argmax(q_hat_arr, axis = 1)
acc_w_adv = np.mean(est_label == true_label_test)
acc_wo_adv = np.mean(est_label[real_test] == true_label_test[real_test])
print '**Final accuracy  w/ adversarial i.p. (only EM):%.5f'%(acc_w_adv)
print '**Final accuracy w/o adversarial i.p. (only EM):%.5f'%(acc_wo_adv)

**Final accuracy  w/ adversarial i.p. (only EM):0.41720
**Final accuracy w/o adversarial i.p. (only EM):0.46384


**Notes**:

1) The accuracy may (no particular pattern seen) decrease after the EM steps. So how many steps to perform is a hyper parameter which has to be tuned carefully. 
    
2) The EM algorithm without the spectral initialization performs as close as the Spectral+EM algorithm. Infact, it sometime outperforms the Spectral+EM algorithm.